#### Modifications for this horrible intcode problem: 

Add relative mode (mode = 2):
- Instead, they count from a value called the relative base. The relative base starts at 0.

The relative base is modified with the relative base offset instruction:

- Opcode 9 adjusts the relative base by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.

In [109]:
def mode_check_1(mode, list_vals, init_val, rel_base):
    if mode == 0:
        return list_vals[init_val + 1]

    if mode == 1:
        return init_val + 1

    if mode == 2:
        return rel_base + list_vals[init_val + 1]

In [110]:
def mode_check_2(mode, list_vals, init_val, rel_base):
    if mode == 0:
        return list_vals[init_val + 2]

    if mode == 1:
        return init_val + 2

    if mode == 2:
        return rel_base + list_vals[init_val + 2]

In [117]:
def intcode_pc(list_vals, init_val, rel_base):
    
    """Built in Day 5"""
    
    while True:
        
        print(list_vals)
        print(rel_base)
        print(init_val)
        # identify the opt_code_str
        opt_code_str = str(list_vals[init_val])

        # take actual opt code: 
        opt_code = int(opt_code_str[-2:])

        # modes can be 0 == position or 1 == immediate 
        # handle mode with try / except
        mode_3, mode_2, mode_1 = 0, 0, 0

        # overwrite 0s UNLESS type error, then we know we had trailing 0s
        # saw example here: https://p.szy.io/MrmgEA/python
        try:
            mode_1 = int(opt_code_str[-3])
            mode_2 = int(opt_code_str[-4])
            mode_3 = int(opt_code_str[-5])

        except IndexError:
            pass


        # handling each type of opt code
        if opt_code == 1:

            # the third param provides the writing point: we always look at the 3rd value after opt code to write
            write_ix = list_vals[init_val + 3]

            # get sum vals
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)
            ans = list_vals[val1] + list_vals[val2]

            # overwrite
            list_vals[write_ix] = ans

            init_val += 4

        elif opt_code == 2:

            write_ix = list_vals[init_val + 3]

            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)
            ans = list_vals[val1] * list_vals[val2]

            # overwrite
            list_vals[write_ix] = ans

            init_val += 4

        elif opt_code == 3:

            write_ix = list_vals[init_val + 1]
            list_vals[write_ix] = int(input())
            init_val += 2


        elif opt_code == 4:
            # output val that is at 'init_val + 1' index: think i need t be able to handle a non-0 val....
            # possibility of mode == 1?
            print("trying 4")
            val1 = mode_check(mode_1, list_vals, init_val, rel_base)
            print(f"Val in 4 is {val1}")
            #print_val = val1 + val2
            print(val1)
            init_val += 2        

        elif opt_code == 5:

            # jump-if-true
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            if list_vals[val1] != 0:
                init_val = list_vals[val2]
            else:
                init_val += 3

        elif opt_code == 6:

            # jump-if-false
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            if list_vals[val1] == 0:
                init_val = list_vals[val2]
            else:
                init_val += 3

        elif opt_code == 7:

            # less than
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            write_ix = list_vals[init_val + 3]

            ans = 1 if list_vals[val1] < list_vals[val2] else 0
            list_vals[write_ix] = ans
            init_val += 4

        elif opt_code == 8:

            # equals
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            write_ix = list_vals[init_val + 3]

            ans = 1 if list_vals[val1] == list_vals[val2] else 0
            list_vals[write_ix] = ans
            init_val += 4
            
        elif opt_code == 9:
            
            # update relative base
            rel_base += list_vals[init_val +1]
            init_val += 2
            
        elif opt_code == 99:
            break

        else:

            print("Something is broken!")

In [118]:
test_case1 = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]

In [119]:
intcode_pc(list_vals = test_case1, init_val = 0, rel_base=0)

[109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99]
0
0
[109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99]
1
2
trying 4
Val in 4 is 204
204
[109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99]
1
4


IndexError: list index out of range

In [43]:
def intcode_calculation(integers, input_values=[1]):
    """
    Intcode program.
    >>> intcode_calculation([1, 0, 0, 0, 99])[0]
    {0: 2, 1: 0, 2: 0, 3: 0, 4: 99}
    >>> intcode_calculation([2, 3, 0, 3, 99])[0]
    {0: 2, 1: 3, 2: 0, 3: 6, 4: 99}
    >>> intcode_calculation([2, 4, 4, 5, 99, 0])[0]
    {0: 2, 1: 4, 2: 4, 3: 5, 4: 99, 5: 9801}
    >>> intcode_calculation([1, 1, 1, 4, 99, 5, 6, 0, 99])[0]
    {0: 30, 1: 1, 2: 1, 3: 4, 4: 2, 5: 5, 6: 6, 7: 0, 8: 99}
    >>> intcode_calculation([3, 0, 4, 0, 99])[0]
    {0: 1, 1: 0, 2: 4, 3: 0, 4: 99}
    >>> intcode_calculation([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9], input_values=[0])[1][0]
    0
    >>> intcode_calculation([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9], input_values=[1])[1][0]
    1
    >>> intcode_calculation([3,3,1105,-1,9,1101,0,0,12,4,12,99,1], input_values=[0])[1][0]
    0
    >>> intcode_calculation([3,3,1105,-1,9,1101,0,0,12,4,12,99,1], input_values=[1])[1][0]
    1
    >>> intcode_calculation([109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99])[1]
    [109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99]
    >>> len(str(intcode_calculation([1102,34915192,34915192,7,4,7,99,0])[1][0]))
    16
    >>> intcode_calculation([104,1125899906842624,99])[1]
    [1125899906842624]
    """
    if isinstance(integers, list):
        integers = convert_to_dict(integers)

    position = 0
    opcode = opcode_decoder(integers.get(position, 0))
    output = []
    relative_base = 0

    while opcode.opcode != 99:
        def get_value(param):
            if opcode.modes[param-1] == 0:
                return integers.get(integers.get(position + param, 0), 0)
            elif opcode.modes[param-1] == 1:
                return integers.get(position + param, 0)
            elif opcode.modes[param-1] == 2:
                return integers.get(relative_base + integers.get(position + param, 0))
            else:
                raise Exception

        def write_position(parameter):
            if opcode.modes[parameter-1] == 0:
                return integers.get(position + parameter, 0)
            elif opcode.modes[parameter-1] == 2:
                return relative_base + integers.get(position + parameter, 0)
            else:
                raise Exception

        if opcode.opcode == 1:
            integers[write_position(3)] = get_value(1) + get_value(2)
            position += 4
        elif opcode.opcode == 2:
            integers[write_position(3)] = get_value(1) * get_value(2)
            position += 4
        elif opcode.opcode == 3:
            integers[write_position(1)] = input_values.pop()
            position += 2
        elif opcode.opcode == 4:
            output.append(get_value(1))
            position += 2
        elif opcode.opcode == 5:
            if get_value(1) != 0:
                position = get_value(2)
            else:
                position += 3
        elif opcode.opcode == 6:
            if get_value(1) == 0:
                position = get_value(2)
            else:
                position += 3
        elif opcode.opcode == 7:
            if get_value(1) < get_value(2):
                integers[write_position(3)] = 1
            else:
                integers[write_position(3)] = 0
            position += 4
        elif opcode.opcode == 8:
            if get_value(1) == get_value(2):
                integers[write_position(3)] = 1
            else:
                integers[write_position(3)] = 0
            position += 4
        elif opcode.opcode == 9:
            relative_base += get_value(1)
            position += 2
        else:
            raise Exception

        opcode = opcode_decoder(integers.get(position, 0))

    return integers, output

In [46]:
def convert_to_dict(integer_list):
    """
    >>> convert_to_dict([1, 2, 3])
    {0: 1, 1: 2, 2: 3}
    """
    return {k: v for k, v in enumerate(integer_list)}

In [47]:
def opcode_decoder(opcode):
    """
    >>> opcode_decoder(1002)
    Opcode(opcode=2, modes=[0, 1, 0])
    >>> opcode_decoder(101)
    Opcode(opcode=1, modes=[1, 0, 0])
    """
    from collections import namedtuple
    Opcode = namedtuple('Opcode', 'opcode modes')

    opcode_full = f'{opcode:05d}'

    return Opcode(opcode=int(opcode_full[3:]),
                  modes=[int(opcode_full[2]), int(opcode_full[1]), int(opcode_full[0])])

In [48]:
intcode_calculation(integers=test_case1, input_values=0)

({0: 109,
  1: 1,
  2: 204,
  3: -1,
  4: 1001,
  5: 100,
  6: 1,
  7: 100,
  8: 1008,
  9: 100,
  10: 16,
  11: 101,
  12: 1006,
  13: 101,
  14: 0,
  15: 99,
  100: 16,
  101: 1},
 [109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99])